In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import *
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from skimage import io, transform
from PIL import Image
import PIL
import cv2

import pandas as pd
import numpy as np

from pathlib import Path
import os 
import random
from statistics import mode
from scipy import stats as s

path = Path("../input/cassava-leaf-disease-classification")

In [ ]:
TTA = True
TTA_samples = 7

In [ ]:
class Dataset_from_dataframe(Dataset):
    """
    Dataset creado teniendo las imágenes en una carpeta y las etiquetas en un 
    csv en el que se mencionan los nombres de las imágenes de entrada a las que 
    corresponde cada etiqueta.
    """
    def __init__(self, path, dataframe, dir_images, transforms=None):
        """
        Args:
            path (Path Object): Ruta del directorio principal
            csv_file (string): Nombre del csv.
            dir_images (string): Ruta a la carpeta con las imágenes.
            transforms (callable, optional): Lista de posibles transformaciones
                hecha con Compose. Debe acabar con ToTensor.
        """
        self.path = path
        self.labels = dataframe
        self.path_images = dir_images
        self.transforms = transforms

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]
        route = self.path/(str(self.path_images) + "/" + str(img_name))
        image = PIL.Image.open(os.path.join(route))
        sample = {"image": image, "name": img_name}

        if self.transforms:
            sample["image"] = self.transforms(sample["image"])
            
        return sample

In [ ]:
!pip install ../input/efficientnet-imports/efficientnet/efficientnet_pytorch-0.7.0/dist/efficientnet_pytorch-0.7.0.tar

In [ ]:
from efficientnet_pytorch import EfficientNet
model = torch.load("../input/model-2/model_2")
model = model.cuda()

In [ ]:
test = pd.DataFrame()
list_files = os.listdir("../input/cassava-leaf-disease-classification/test_images")
test['image_id'] = pd.Series(list_files)
test['label'] = 0
#test["result"] = 0
test = test
test

In [ ]:
size1 = [600, 800]

current_size = size1

if TTA:
    tsfm_TTA = transforms.Compose([
        transforms.Resize(current_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize((0.4307, 0.4967, 0.3135), (0.1960, 0.2011, 0.1767))
    ])
    to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4307, 0.4967, 0.3135), (0.1960, 0.2011, 0.1767))
    ])
    tsfm = None
else:
    tsfm = transforms.Compose([
            transforms.Resize(current_size),
            transforms.ToTensor(),
            transforms.Normalize((0.4307, 0.4967, 0.3135), (0.1960, 0.2011, 0.1767))
        ])


folder = "test_images"
test_dataset = Dataset_from_dataframe(path, test, folder, tsfm)

In [ ]:
b_size = 1

test_dataloader = DataLoader(test_dataset, batch_size=b_size, shuffle=False, num_workers=0)

In [ ]:
model = model.eval()

if TTA:
    with torch.no_grad():
        for sample in test_dataset:
            results = []
            image = sample["image"]
            for i in range(TTA_samples):
                if i == 0: X_batch = to_tensor(image).unsqueeze(0).to("cuda:0")
                else: X_batch = tsfm_TTA(image).unsqueeze(0).to("cuda:0")
                y_pred = model(X_batch)
                results.append(y_pred.argmax(axis=1).item())
            
            voted_result = s.mode(results)[0].item()
            test.loc[test.image_id == sample["name"], "label"] = voted_result
else:
    with torch.no_grad():
        for sample in iter(test_dataloader):
            X_batch = sample["image"].to("cuda:0")
            y_pred = model(X_batch)
            test.loc[test.image_id == sample["name"][0], "label"] = y_pred.argmax(axis=1).item()

In [ ]:
test.to_csv('submission.csv',index=False)